In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#The purpose of this book is to compare in one plot the results of varying parameters.

geoM2xy_100nm_PS_in_0_4_agarose = np.genfromtxt('./100nm_PS-NPs/geoM2xy_100nm_PS_in_0-4_agarose.csv',delimiter =",")
geoM2xy_100nm_PS_PEG_in_0_4_agarose = np.genfromtxt('./100nm_PS-PEG/geoM2xy_100nm_PS_PEG_in_0-4_agarose.csv',delimiter =",")
# geoM2xy_20nm_PS_in_0_4_agarose = np.genfromtxt('./20nm_PS-NPs/geoM2xy_20nm_PS_in_0-4_agarose',delimiter =",")
# geoM2xy_20nm_PS_PEG_in_0_4_agarose = np.genfromtxt('./20nm_PS-PEG/geoM2xy_20nm_PS_PEG_in_0-4_agarose',delimiter =",")

In [43]:
geoD2xy_100nm_PS_in_0_4_agarose = np.genfromtxt('./100nm_PS-NPs/geoD2xy_100nm_PS_in_0-4_agarose.csv',delimiter =",")
geoD2xy_100nm_PS_PEG_in_0_4_agarose = np.genfromtxt('./100nm_PS-PEG/geoD2xy_100nm_PS_PEG_in_0-4_agarose.csv',delimiter =",")

In [3]:
# geoM2xy_camera_1000ms_60x = np.genfromtxt('./geoM2xy_camera_1000ms_60x.csv',delimiter =",")
# arM2xy_camera_1000ms_60x = np.genfromtxt('./arM2xy_camera_1000ms_60x.csv',delimiter =",")

In [4]:
f1 = 0.5
f2 = 3.95

time1 = np.zeros(geoM2xy_100nm_PS_in_0_4_agarose.shape[0])
time2 = np.zeros(geoM2xy_100nm_PS_PEG_in_0_4_agarose.shape[0])

for num in range(0, geoM2xy_100nm_PS_in_0_4_agarose.shape[0]):
    time1[num] = num/f1

for num in range(0, geoM2xy_100nm_PS_PEG_in_0_4_agarose.shape[0]):
    time2[num] = num/f2

In [46]:
time2

array([  0.        ,   0.25316456,   0.50632911,   0.75949367,
         1.01265823,   1.26582278,   1.51898734,   1.7721519 ,
         2.02531646,   2.27848101,   2.53164557,   2.78481013,
         3.03797468,   3.29113924,   3.5443038 ,   3.79746835,
         4.05063291,   4.30379747,   4.55696203,   4.81012658,
         5.06329114,   5.3164557 ,   5.56962025,   5.82278481,
         6.07594937,   6.32911392,   6.58227848,   6.83544304,
         7.08860759,   7.34177215,   7.59493671,   7.84810127,
         8.10126582,   8.35443038,   8.60759494,   8.86075949,
         9.11392405,   9.36708861,   9.62025316,   9.87341772,
        10.12658228,  10.37974684,  10.63291139,  10.88607595,
        11.13924051,  11.39240506,  11.64556962,  11.89873418,
        12.15189873,  12.40506329,  12.65822785,  12.91139241,
        13.16455696,  13.41772152,  13.67088608,  13.92405063,
        14.17721519,  14.43037975,  14.6835443 ,  14.93670886,
        15.18987342,  15.44303797,  15.69620253,  15.94

In [45]:
geoD2xy_100nm_PS_PEG_in_0_4_agarose

array([         nan,   5.81344562,   5.61276853,   5.27161267,
         4.91161979,   4.45350733,   4.40961589,   4.29199192,
         4.25395254,   4.22844102,   4.24340597,   4.19525438,
         4.17695695,   4.16170802,   4.09717313,   4.01649121,
         3.92095056,   3.95203222,   3.73298069,   3.70485137,
         3.5795085 ,   3.53361355,   3.45978495,   3.3441739 ,
         3.2708956 ,   3.31492936,   3.28883084,   3.37009337,
         3.36039668,   3.31492408,   3.41478373,   3.47899024,
         3.3298594 ,   3.18953492,   3.18189888,   3.04863592,
         3.02114645,   3.06722662,   3.13230882,   3.1198247 ,
         2.88558663,   2.73407971,   2.62329584,   2.46465693,
         2.29135856,   2.27652205,   2.41836314,   2.44196207,
         2.39600848,   2.19264144,   2.07984528,   2.03981153,
         1.64571258,   1.7264948 ,   1.67741171,   1.46009924,
         1.6226996 ,   1.43779368,   1.3930014 ,   1.35142722,
         1.31627064,   1.31168505,   1.13254159,   1.16

In [6]:
#Linear algebra to find Deff (PS-PEG-NPs):
to_frame1 = 17

w = dict()
residuals = dict()
line = dict()
A1 = np.ones((np.shape(time2)[0], 2))
A1[:, 0] = time2

w[0], residuals[0] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_100nm_PS_PEG_in_0_4_agarose[1:to_frame1])[0:2]
# w[1] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_20nm_PS_PEG_in_0_4_agarose[1:to_frame1])[0]

line[0] = w[0][0]*time2 + w[0][1]
# line[1] = w[1][0]*time2 + w[1][1]

In [7]:
print('Diffusion Coeff (um^2/s):', np.round(w[0][0], 7), 'geoM2xy_100nm_PS_PEG_in_0_4_agarose')
# print('Diffusion Coeff (um^2/s):', np.round(w[1][0], 7), 'geoM2xy_20nm_PS_PEG_in_0_4_agarose')

Diffusion Coeff (um^2/s): 3.8130148 geoM2xy_100nm_PS_PEG_in_0_4_agarose


In [8]:
residuals[0]/17

array([ 0.03883351])

In [9]:
#Linear algebra to find Deff (PS NPs):
to_frame = 20

w1 = dict()
residuals1 = dict()
line1 = dict()
A2 = np.ones((np.shape(time1)[0], 2))
A2[:, 0] = time1


w1[0], residuals1[0] = np.linalg.lstsq(A2[1:to_frame, :], geoM2xy_100nm_PS_in_0_4_agarose[1:to_frame])[0:2]
# w1[1] = np.linalg.lstsq(A2[1:to_frame, :], geoM2xy_20nm_PS_in_0_4_agarose[1:to_frame])[0]

line1[0] = w1[0][0]*time1 + w1[0][1]
# line1[1] = w1[1][0]*time1 + w1[1][1]

In [10]:
print('Diffusion Coeff (um^2/s):', np.round(w1[0][0], 7), 'geoM2xy_100nm_PS_in_0_4_agarose')
# print('Diffusion Coeff (um^2/s):', np.round(w1[1][0], 7), 'geoM2xy_20nm_PS_in_0_4_agarose')

Diffusion Coeff (um^2/s): 1.57e-05 geoM2xy_100nm_PS_in_0_4_agarose


In [11]:
residuals1[0]/20

array([  8.60502435e-10])

In [12]:
# N = 5

# ind = np.arange(N)
# width = 0.35     

# x1 = [1, 2, 3, 4]
# x2 = [1, 2, 3, 4]
# y1 = [w[0][0], w[1][0], w1[0][0], w1[1][0]]
# y2 = [w[2][0], w[3][0], w1[2][0], w1[3][0]]

# labels1 = ['FM2xy_camera_1000ms_40x', 'FM2xy_camera_1000ms_60x', 'FM2xy_camera_625ms_40x', 'FM2xy_camera_625ms_60x']
# labels2 = ['FM2xy_confocal_1000ms_40x', 'FM2xy_confocal_1000ms_60x', 'FM2xy_confocal_625ms_40x', 'FM2xy_confocal_625ms_60x']

In [13]:
# fig = plt.figure()
# ax = fig.add_subplot(111)

# ## the data
# N = 4

# ## necessary variables
# ind = np.arange(N)                # the x locations for the groups
# width = 0.6                     # the width of the bars

# ## the bars
# rects1 = ax.bar(ind, y1, width,
#                 color='green')

# # axes and labels
# ax.set_xlim(-width/2,len(ind)+width/2)
# ax.set_ylim(0,0.0012)
# ax.set_ylabel('Diffusion Coefficient (um^2/s)')
# ax.set_title('Diffusion coefficients for camera')
# xTickMarks = [labels1[i-1] for i in range(1,5)]
# ax.set_xticks(ind+width)
# xtickNames = ax.set_xticklabels(xTickMarks)
# plt.setp(xtickNames, rotation=45, fontsize=10)

# plt.show()

In [14]:
# fig = plt.figure()
# ax = fig.add_subplot(111)

# ## the data
# N = 4

# ## necessary variables
# ind = np.arange(N)                # the x locations for the groups
# width = 0.6                     # the width of the bars

# ## the bars
# rects1 = ax.bar(ind, y2, width,
#                 color='green')

# # axes and labels
# ax.set_xlim(-width/2,len(ind)+width/2)
# ax.set_ylim(0,0.0012)
# ax.set_ylabel('Diffusion Coefficient (um^2/s)')
# ax.set_title('Diffusion coefficients for confocal')
# xTickMarks = [labels2[i-1] for i in range(1,5)]
# ax.set_xticks(ind+width)
# xtickNames = ax.set_xticklabels(xTickMarks)
# plt.setp(xtickNames, rotation=45, fontsize=10)

# plt.show()

In [40]:
#Figure parameters

filename = 'geo_comparison.png'

limit1=8.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=20.1 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'<MSD> ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=0 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [16]:
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"
    return "#" + "".join([hex(i)[2:] for i in new_rgb_int])

In [17]:
# for name, hex in mpl.colors.cnames.items():
#     print(name, hex)

g2 = color_variant('#008000', brightness_offset=0)
b2 = color_variant('#0000FF', brightness_offset=0)

In [18]:
def clamp(val, minimum=0, maximum=255):
    if val < minimum:
        return minimum
    if val > maximum:
        return maximum
    return val

def colorscale(hexstr, scalefactor):
    """
    Scales a hex string by ``scalefactor``. Returns scaled hex string.

    To darken the color, use a float value between 0 and 1.
    To brighten the color, use a float value greater than 1.

    >>> colorscale("#DF3C3C", .5)
    #6F1E1E
    >>> colorscale("#52D24F", 1.6)
    #83FF7E
    >>> colorscale("#4F75D2", 1)
    #4F75D2
    """

    hexstr = hexstr.strip('#')

    if scalefactor < 0 or len(hexstr) != 6:
        return hexstr

    r, g, b = int(hexstr[:2], 16), int(hexstr[2:4], 16), int(hexstr[4:], 16)

    r = round(clamp(r * scalefactor))
    g = round(clamp(g * scalefactor))
    b = round(clamp(b * scalefactor))

    return "#%02x%02x%02x" % (r, g, b)

In [19]:
g2 = colorscale('#008000', 1.5)
b2 = colorscale('#0000FF', 1.5)
# round(clamp(255*0.1))
g2

'#00c000'

In [42]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS')
ax.plot(time2[0:55], geoM2xy_100nm_PS_PEG_in_0_4_agarose[0:55], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PS-PEG')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time2[1:55], line[0][1:55], linewidth=wi-3, color=g2)
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.01, 0.86), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [21]:
line1[0]

array([ 0.00078031,  0.00081163,  0.00084295,  0.00087428,  0.0009056 ,
        0.00093692,  0.00096824,  0.00099956,  0.00103088,  0.0010622 ,
        0.00109352,  0.00112485,  0.00115617,  0.00118749,  0.00121881,
        0.00125013,  0.00128145,  0.00131277,  0.0013441 ,  0.00137542,
        0.00140674,  0.00143806,  0.00146938,  0.0015007 ,  0.00153202,
        0.00156335,  0.00159467,  0.00162599,  0.00165731,  0.00168863,
        0.00171995,  0.00175127,  0.0017826 ,  0.00181392,  0.00184524,
        0.00187656,  0.00190788,  0.0019392 ,  0.00197052,  0.00200184,
        0.00203317,  0.00206449,  0.00209581,  0.00212713,  0.00215845,
        0.00218977,  0.00222109,  0.00225242,  0.00228374,  0.00231506,
        0.00234638,  0.0023777 ,  0.00240902,  0.00244034,  0.00247167,
        0.00250299,  0.00253431,  0.00256563,  0.00259695,  0.00262827,
        0.00265959,  0.00269092,  0.00272224,  0.00275356,  0.00278488,
        0.0028162 ,  0.00284752,  0.00287884,  0.00291017,  0.00

In [22]:
geoM2xy_100nm_PS_PEG_in_0_4_agarose

array([             nan,   1.47175839e+00,   2.84190812e+00,
         4.00375646e+00,   4.97379219e+00,   5.63735105e+00,
         6.69815072e+00,   7.60606163e+00,   8.61560008e+00,
         9.63442257e+00,   1.07427999e+01,   1.16829869e+01,
         1.26894895e+01,   1.36967606e+01,   1.45216263e+01,
         1.52524983e+01,   1.58823314e+01,   1.70087463e+01,
         1.70110513e+01,   1.78208040e+01,   1.81240937e+01,
         1.87862999e+01,   1.92696883e+01,   1.94724050e+01,
         1.98737960e+01,   2.09805656e+01,   2.16480005e+01,
         2.30360813e+01,   2.38205334e+01,   2.43374173e+01,
         2.59350663e+01,   2.73034677e+01,   2.69760761e+01,
         2.66467475e+01,   2.73884967e+01,   2.70132297e+01,
         2.75344993e+01,   2.87309836e+01,   3.01336038e+01,
         3.08033325e+01,   2.92211305e+01,   2.83790553e+01,
         2.78932723e+01,   2.68304425e+01,   2.55239941e+01,
         2.59350613e+01,   2.81632163e+01,   2.90562575e+01,
         2.91160525e+01,

In [23]:
# #Figure parameters

# filename = 'FBF_comparison_60x.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [24]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_60x, 'bo', ms=mi, label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'go', ms=mi, label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [25]:
# #Figure parameters

# filename = 'FBF_comparison_625ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [26]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, FM2xy_camera_625ms_40x, 'bo', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [27]:
# #Figure parameters

# filename = 'FBF_comparison_1000ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [28]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [29]:
# #Figure parameters

# filename = 'FBF_comparison_camera.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [30]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_40x, 'go', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [31]:
# #Figure parameters

# filename = 'FBF_comparison_confocal.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [32]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'bo', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time2, line[2], linewidth=3, color='purple')
# ax.plot(time2, line[3], linewidth=3, color='purple')
# ax.plot(time1, line1[2], linewidth=3, color='purple')
# ax.plot(time1, line1[3], linewidth=3, color='purple')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [33]:
# #Figure parameters

# filename = 'FBF_geo_ar_comparison.png'

# limit1=20.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.0051 #y range
# tick2 = round(limit2, 3)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=3 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [34]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2[0:19], geoM2xy_camera_1000ms_60x[0:-1], 'b--', linewidth=wi, dashes=[d1,d2], label='Geometric')
# ax.plot(time2[0:19], arM2xy_camera_1000ms_60x[0:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Arithmetic')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'r--', linewidth=wi, dashes=[d1,d2], label='Frame-by-frame')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()